In [2]:
# Import packages and load file
import pandas as pd
import pandasql as ps
pysql = lambda q: ps.sqldf(q, globals())

file = 'Adops & Data Scientist Sample Data.xlsx'

# Load file
xls = pd.ExcelFile(file)

# Check if file loaded properly
print(xls.sheet_names)

['Q1 Analytics', 'Q2 Regression']


In [3]:
# Load data to dataframe
df_analytics = xls.parse('Q1 Analytics', skiprows = 0)

# First look at data
print(df_analytics.head())
print(df_analytics.shape)

                    ts user_id country_id site_id
0  2019-02-01 00:01:24  LC36FC        TL6   N0OTG
1  2019-02-01 00:10:19  LC39B6        TL6   N0OTG
2  2019-02-01 00:21:50  LC3500        TL6   N0OTG
3  2019-02-01 00:22:50  LC374F        TL6   N0OTG
4  2019-02-01 00:23:44  LCC1C3        TL6   QGO3G
(3553, 4)


In [7]:
df_analytics_BDV = df_analytics[df_analytics['country_id'] == "BDV"]

print(df_analytics_BDV.head())
print(df_analytics_BDV.shape)

# Unique site_id for BDV
set_siteId_BDV = df_analytics_BDV['site_id'].unique()
print(set_siteId_BDV)

#For each site_id, number of unique user_id's in 844 rows.
for sid in set_siteId_BDV:
    print(sid, df_analytics_BDV[df_analytics_BDV['site_id'] == sid]['user_id'].unique().shape)

                     ts user_id country_id site_id
13  2019-02-01 00:47:58  LC3C22        BDV   N0OTG
32  2019-02-01 02:46:15  LC375A        BDV   5NPAU
87  2019-02-01 06:44:42  LC392E        BDV   N0OTG
88  2019-02-01 07:36:56  LC3FAE        BDV   5NPAU
94  2019-02-01 08:58:35  LC3AFF        BDV   3POLC
(844, 4)
['N0OTG' '5NPAU' '3POLC']
N0OTG (90,)
5NPAU (544,)
3POLC (2,)


In [12]:
# Data between 2019-02-03 00:00:00 and 2019-02-04 23:59:59
df_analytics_timeslice = df_analytics[(df_analytics['ts'] >= '2019-02-03 00:00:00') & (df_analytics['ts'] <= '2019-02-04 23:59:59')] 

# Unique site_id for original dataset
set_siteId = df_analytics['site_id'].unique()
print(set_siteId)

# Sites visited more than 10 times
for sid in set_siteId:
    df_site_visitFreq = df_analytics_timeslice[df_analytics_timeslice['site_id'] == sid].groupby('user_id').count()['site_id'].reset_index(name="count")   
    print(sid) 
    print(df_site_visitFreq[df_site_visitFreq['count'] >= 10])

['N0OTG' 'QGO3G' 'GVOFK' '3POLC' '5NPAU' 'RT9Z6' 'JSUUP' 'EUZ/Q']
N0OTG
    user_id  count
0    LC06C3     25
150  LC3A59     26
171  LC3C9D     17
QGO3G
Empty DataFrame
Columns: [user_id, count]
Index: []
GVOFK
Empty DataFrame
Columns: [user_id, count]
Index: []
3POLC
  user_id  count
3  LC3C7E     15
5NPAU
Empty DataFrame
Columns: [user_id, count]
Index: []
RT9Z6
Empty DataFrame
Columns: [user_id, count]
Index: []
JSUUP
Empty DataFrame
Columns: [user_id, count]
Index: []
EUZ/Q
Empty DataFrame
Columns: [user_id, count]
Index: []


In [15]:
q1 = """SELECT * FROM df_analytics where user_id == 'LC3561' """
q2 = """SELECT * from (SELECT site_id, user_id, max(ts) FROM df_analytics GROUP BY user_id) where user_id == 'LC3561'"""
q3 = """SELECT site_id, user_id, max(ts) FROM df_analytics GROUP BY user_id"""

# Query: All sites, based on metric of last visited site
q4 = """SELECT site_id, COUNT(user_id) AS total_users FROM (SELECT site_id, user_id, MAX(ts) FROM df_analytics GROUP BY user_id) GROUP BY site_id ORDER BY total_users DESC"""

df1 = pysql(q1)
df2 = pysql(q2)
df3 = pysql(q4)

# All sites, based on metric of last visited site
print(df3)
df3.shape

# Top 3 sites, based on metric of last visited site
print(df3[:3])

  site_id  total_users
0   5NPAU          992
1   N0OTG          561
2   QGO3G          289
3   GVOFK           42
4   3POLC           28
5   RT9Z6            2
6   EUZ/Q            1
7   JSUUP            1
  site_id  total_users
0   5NPAU          992
1   N0OTG          561
2   QGO3G          289


In [23]:
q_firstsite = """SELECT user_id, site_id AS site_id_first, min(ts) AS first_visit_ts FROM df_analytics GROUP BY user_id"""
q_lastsite = """SELECT user_id, site_id AS site_id_last, max(ts) AS last_visit_ts FROM df_analytics GROUP BY user_id"""
q5_check = """SELECT * FROM df_analytics where user_id == 'LC06C3' """

df_firstsite = pysql(q_firstsite)
df_lastsite = pysql(q_lastsite)
df3 = pysql(q5_check)
q_sitevisits_firstlast = """SELECT df_firstsite.user_id, df_firstsite.site_id_first, df_lastsite.site_id_last, df_firstsite.first_visit_ts, df_lastsite.last_visit_ts FROM df_firstsite INNER JOIN df_lastsite ON df_firstsite.user_id = df_lastsite.user_id"""
df_sitevisits_firstlast = pysql(q_sitevisits_firstlast)
df_sitevisits_firstlast['same_site_firstlast'] = (df_sitevisits_firstlast['site_id_first'] == df_sitevisits_firstlast['site_id_last'])
# All sites, based on metric of last visited site
print(df_firstsite.head())
print(df_firstsite.shape)

print(df_lastsite.head())
print(df_lastsite.shape)

print(df_sitevisits_firstlast.head())
print(df_sitevisits_firstlast.shape)

print(df_sitevisits_firstlast[df_sitevisits_firstlast['same_site_firstlast'] == True].shape)

  user_id site_id_first       first_visit_ts
0  LC00C3         5NPAU  2019-02-03 18:52:50
1  LC01C3         5NPAU  2019-02-04 11:35:10
2  LC05C3         5NPAU  2019-02-02 14:14:44
3  LC06C3         N0OTG  2019-02-01 22:49:39
4  LC07C3         5NPAU  2019-02-05 19:06:42
(1916, 3)
  user_id site_id_last        last_visit_ts
0  LC00C3        5NPAU  2019-02-03 18:52:50
1  LC01C3        5NPAU  2019-02-04 11:35:10
2  LC05C3        5NPAU  2019-02-02 14:14:44
3  LC06C3        N0OTG  2019-02-07 01:16:12
4  LC07C3        5NPAU  2019-02-05 19:06:42
(1916, 3)
  user_id site_id_first site_id_last       first_visit_ts  \
0  LC00C3         5NPAU        5NPAU  2019-02-03 18:52:50   
1  LC01C3         5NPAU        5NPAU  2019-02-04 11:35:10   
2  LC05C3         5NPAU        5NPAU  2019-02-02 14:14:44   
3  LC06C3         N0OTG        N0OTG  2019-02-01 22:49:39   
4  LC07C3         5NPAU        5NPAU  2019-02-05 19:06:42   

         last_visit_ts  same_site_firstlast  
0  2019-02-03 18:52:50           